In [1]:
%load_ext autoreload
%autoreload 2
#imports
from aiflows.utils.general_helpers import read_yaml_file, quick_load_api_keys
from aiflows.utils import serve_utils
from aiflows.utils import colink_utils
from aiflows.workers import run_dispatch_worker_thread
from aiflows.base_flows import AtomicFlow
from aiflows.messages import FlowMessage
from aiflows import flow_verse
import sys
import os
sys.path.append("..")
from utils import compile_and_writefile, dict_to_yaml
import json
import copy
#Specify path of your flow modules
FLOW_MODULES_PATH = "./"

/Users/nicolasbaldwin/opt/miniconda3/envs/coflows/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Connect to the CoLink Server

In [2]:
cl = colink_utils.start_colink_server()

## 2. ChatFlow

### 2.1 Importing ChatFlow From the FlowVerse

In [3]:
#import the ChatFlowModule from the flowverse on hugging face
# link: https://huggingface.co/aiflows/ChatFlowModule/tree/coflows
dependencies = [
    {"url": "aiflows/ChatFlowModule", "revision": "coflows"},
]
#Dowload flow modules in "./flow_modules"
flow_verse.sync_dependencies(dependencies)


[2024-03-18 16:43:39,963][aiflows.flow_verse.loading:775][INFO] - [<interactive>] started to sync flow module dependencies to /Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/aiflow-colink/aiflows/AMLD/ChatFlow/flow_modules...
[2024-03-18 16:43:40,391][aiflows.flow_verse.loading:608][INFO] - aiflows/ChatFlowModule:coflows already synced, skip
[2024-03-18 16:43:40,393][aiflows.flow_verse.loading:825][INFO] - [<interactive>] finished syncing




['/Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/aiflow-colink/aiflows/AMLD/ChatFlow/flow_modules/aiflows/ChatFlowModule']

In [4]:
## Print the configuration
default_cfg = read_yaml_file("flow_modules/aiflows/ChatFlowModule/ChatAtomicFlow.yaml")
print(json.dumps(default_cfg, indent=4))


{
    "_target_": "flow_modules.aiflows.ChatFlowModule.ChatAtomicFlow.instantiate_from_default_config",
    "name": "ChatAtomicFlow",
    "description": "Flow which uses as tool an LLM though an API",
    "enable_cache": true,
    "n_api_retries": 6,
    "wait_time_between_retries": 20,
    "system_name": "system",
    "user_name": "user",
    "assistant_name": "assistant",
    "backend": {
        "_target_": "aiflows.backends.llm_lite.LiteLLMBackend",
        "api_infos": "???",
        "model_name": "gpt-3.5-turbo",
        "n": 1,
        "max_tokens": 2000,
        "temperature": 0.3,
        "top_p": 0.2,
        "frequency_penalty": 0,
        "presence_penalty": 0,
        "stream": true
    },
    "system_message_prompt_template": {
        "_target_": "aiflows.prompt_template.JinjaPrompt"
    },
    "init_human_message_prompt_template": {
        "_target_": "aiflows.prompt_template.JinjaPrompt"
    },
    "human_message_prompt_template": {
        "_target_": "aiflows.prompt

### 2.2 Serving & Getting and Instance of the ChatFlow

#### 2.2.1 Serving the ChatFlow

In [5]:
serve_utils.serve_flow(
    cl=cl,
    flow_class_name="flow_modules.aiflows.ChatFlowModule.ChatAtomicFlow",
    flow_endpoint="Chat Flow",
)

Started serving flow_modules.aiflows.ChatFlowModule.ChatAtomicFlow at flows:Chat Flow.
dispatch_point: coflows_dispatch
parallel_dispatch: False
singleton: False


True

#### 2.2.1 Getting 2 Instances of the ChatFlow: One who answers in English and One who answers in French

In [6]:
# Start a worker thread to handle incoming messages
run_dispatch_worker_thread(cl)


Dispatch worker started in attached thread.
dispatch_point: coflows_dispatch


In [7]:
#get the demo config as a starting point
root_dir = "."
cfg_path = os.path.join(root_dir, "flow_modules/aiflows/ChatFlowModule/demo.yaml")
cfg = read_yaml_file(cfg_path)
print(json.dumps(cfg, indent=4))

{
    "_target_": "flow_modules.aiflows.ChatFlowModule.ChatAtomicFlow.instantiate_from_default_config",
    "name": "SimpleQA_Flow",
    "description": "A flow that answers questions.",
    "input_interface_non_initialized": [
        "question"
    ],
    "backend": {
        "_target_": "aiflows.backends.llm_lite.LiteLLMBackend",
        "api_infos": "???",
        "model_name": {
            "openai": "gpt-3.5-turbo",
            "azure": "azure/gpt-4"
        },
        "n": 1,
        "max_tokens": 3000,
        "temperature": 0.3,
        "top_p": 0.2,
        "frequency_penalty": 0,
        "presence_penalty": 0
    },
    "n_api_retries": 6,
    "wait_time_between_retries": 20,
    "system_message_prompt_template": {
        "_target_": "aiflows.prompt_template.JinjaPrompt",
        "template": "You are a helpful chatbot that truthfully answers questions.",
        "input_variables": [],
        "partial_variables": {}
    },
    "init_human_message_prompt_template": {
        

In [8]:
#Get an instance of a the ChatFlow who speaks English
from aiflows.backends.api_info import ApiInfo
english_chatbot_overrides = copy.deepcopy(cfg)

api_info = [ApiInfo(backend_used="openai", api_key=os.getenv("OPENAI_API_KEY"))]
quick_load_api_keys(english_chatbot_overrides, api_info)

english_chatbot_overrides["system_message_prompt_template"]["template"] = \
"You are a helpful chatbot that truthfully answers questions. Answer in the following language: English."

english_chatbot = serve_utils.get_flow_instance(
    cl=cl,
    flow_endpoint="Chat Flow",
    config_overrides=english_chatbot_overrides,
)

#Get an instance of a the ChatFlow who speaks French
french_chatbot_overrides = copy.deepcopy(cfg)
api_info = [ApiInfo(backend_used="openai", api_key=os.getenv("OPENAI_API_KEY"))]
quick_load_api_keys(french_chatbot_overrides, api_info)
french_chatbot_overrides["system_message_prompt_template"]["template"] = \
"You are a helpful chatbot that truthfully answers questions. Answer in the following language: French."

french_chatbot = serve_utils.get_flow_instance(
    cl=cl,
    flow_endpoint="Chat Flow",
    config_overrides=french_chatbot_overrides,
)

Mounted 772025d8-d7d4-4e61-a2f2-3623152f7354 at flows:Chat Flow:mounts:local:772025d8-d7d4-4e61-a2f2-3623152f7354
Mounted f5ac3956-421d-46a4-bd53-26036af29224 at flows:Chat Flow:mounts:local:f5ac3956-421d-46a4-bd53-26036af29224


### 2.3 Calling the ChatFlow

#### 2.3.1 Chatting with the English ChatFlow

In [9]:
input_message1 = FlowMessage(
    data={"id": 0, "question": "What is the capital of Switzerland?"},
)

input_message2 = FlowMessage(
    data={"id": 0, "query": "Where is it located?"},
)

messages = [input_message1, input_message2]

for msg in messages:
    print("~~~~Sent message~~~~\n", msg.data)
    future = english_chatbot.get_reply_future(msg)
    reply = future.get_data()
    print("~~~~Reply~~~~ \n",reply)
    


~~~~Sent message~~~~
 {'id': 0, 'question': 'What is the capital of Switzerland?'}



~~~ Dispatch task ~~~
flow_endpoint: Chat Flow
flow_id: 772025d8-d7d4-4e61-a2f2-3623152f7354
owner_id: local
message_paths: ['push_tasks:4042fc7a-b7ec-4e08-b288-b6d5dbac753b:msg']
parallel_dispatch: False

Input message source: Proxy_Chat Flow

~~~ Dispatch task ~~~
flow_endpoint: Chat Flow
flow_id: 772025d8-d7d4-4e61-a2f2-3623152f7354
owner_id: local
message_paths: ['push_tasks:6131aafa-3252-488c-9466-f8986202d751:msg']
parallel_dispatch: False

Input message source: Proxy_Chat Flow

~~~ Dispatch task ~~~
flow_endpoint: Chat Flow
flow_id: f5ac3956-421d-46a4-bd53-26036af29224
owner_id: local
message_paths: ['push_tasks:ebe5e1a0-bd1b-4403-baa4-f50330a86bc2:msg']
parallel_dispatch: False

Input message source: Proxy_Chat Flow

~~~ Dispatch task ~~~
flow_endpoint: Chat Flow
flow_id: f5ac3956-421d-46a4-bd53-26036af29224
owner_id: local
message_paths: ['push_tasks:25aa6e1f-9366-4639-b2ed-e44d6e16ad16:msg']
parallel_dispatch: False

Input message source: Proxy_Chat Flow

~~~ Dispatch task ~

~~~~Reply~~~~ 
 {'api_output': 'The capital of Switzerland is Bern.'}
~~~~Sent message~~~~
 {'id': 0, 'query': 'Where is it located?'}
~~~~Reply~~~~ 
 {'api_output': 'Bern is located in the central part of Switzerland, on the Swiss Plateau.'}


#### 2.3.2 Chatting with the French ChatFlow

##

In [10]:

for msg in messages:
    print("~~~~Sent message~~~~\n", msg.data)
    future = french_chatbot.get_reply_future(msg)
    reply = future.get_data()
    print("~~~~Reply~~~~ \n",reply)



~~~~Sent message~~~~
 {'id': 0, 'question': 'What is the capital of Switzerland?'}


~~~~Reply~~~~ 
 {'api_output': 'La capitale de la Suisse est Berne.'}
~~~~Sent message~~~~
 {'id': 0, 'query': 'Where is it located?'}
~~~~Reply~~~~ 
 {'api_output': 'Berne est située dans le centre de la Suisse, sur les rives de la rivière Aar.'}


### 2.4 Change Model

In [11]:
english_chatbot_gpt4_overrides = copy.deepcopy(cfg)

english_chatbot_gpt4_overrides["backend"]["model_name"] = {"openai": "gpt-4"}

api_info = [ApiInfo(backend_used="openai", api_key=os.getenv("OPENAI_API_KEY"))]
quick_load_api_keys(english_chatbot_gpt4_overrides, api_info)



english_chatbot_gpt4 = serve_utils.get_flow_instance(
    cl=cl,
    flow_endpoint="Chat Flow",
    config_overrides=english_chatbot_gpt4_overrides,
)




Mounted ce9fc95d-6c90-46b1-bea2-f3275de2cb16 at flows:Chat Flow:mounts:local:ce9fc95d-6c90-46b1-bea2-f3275de2cb16


In [12]:
input_message1 = FlowMessage(
    data={"id": 0, "question": "What is the capital of Switzerland?"},
)

input_message2 = FlowMessage(
    data={"id": 0, "query": "Where is it located?"},
)

messages = [input_message1, input_message2]

for msg in messages:
    print("~~~~Sent message~~~~\n", msg.data)
    future = english_chatbot_gpt4.get_reply_future(msg)
    reply = future.get_data()
    print("~~~~Reply~~~~ \n",reply)

~~~~Sent message~~~~
 {'id': 0, 'question': 'What is the capital of Switzerland?'}


~~~~Reply~~~~ 
 {'api_output': 'The capital of Switzerland is Bern.'}
~~~~Sent message~~~~
 {'id': 0, 'query': 'Where is it located?'}
~~~~Reply~~~~ 
 {'api_output': "Bern is located in the west-central part of Switzerland. It's situated on a peninsula formed by the meandering waters of the River Aare."}


### 2.4 Do it yourself: Mount a ChatFlow and personalize it (Actions required)


- Option 1: Mount a Chatflow who generates text with a `temperature = 1` and always ends replies with "my good friend"
    - Hint: To format the reply, play around with `system_message_prompt_template`

- Option 2: Personalize your flow as you like !

In [13]:
# This is what the demo config looks like ! Modify it to your needs
print(json.dumps(cfg, indent=4))

{
    "_target_": "flow_modules.aiflows.ChatFlowModule.ChatAtomicFlow.instantiate_from_default_config",
    "name": "SimpleQA_Flow",
    "description": "A flow that answers questions.",
    "input_interface_non_initialized": [
        "question"
    ],
    "backend": {
        "_target_": "aiflows.backends.llm_lite.LiteLLMBackend",
        "api_infos": "???",
        "model_name": {
            "openai": "gpt-3.5-turbo",
            "azure": "azure/gpt-4"
        },
        "n": 1,
        "max_tokens": 3000,
        "temperature": 0.3,
        "top_p": 0.2,
        "frequency_penalty": 0,
        "presence_penalty": 0
    },
    "n_api_retries": 6,
    "wait_time_between_retries": 20,
    "system_message_prompt_template": {
        "_target_": "aiflows.prompt_template.JinjaPrompt",
        "template": "You are a helpful chatbot that truthfully answers questions.",
        "input_variables": [],
        "partial_variables": {}
    },
    "init_human_message_prompt_template": {
        

In [20]:
# Deepcopy of demo config
overrides_config = copy.deepcopy(cfg)

#TODO: Mount a Chatflow who generates text with a `temperature = 1` and a `system_message_prompt_template` that is personalized
overrides_config["backend"]["temperature"] = 1.0
overrides_config["system_message_prompt_template"]["template"] = \
    "You are a helpful chatbot that fills out a given prompt in areas surrounded by brackets '[[]]'. Within the brackets are instructions you should follow."


#loading api key to config
api_info = [ApiInfo(backend_used="openai", api_key=os.getenv("OPENAI_API_KEY"))]
quick_load_api_keys(overrides_config, api_info)


#TO DO get a flow instance
personalized_chatbot = serve_utils.get_flow_instance(
    cl=cl,
    flow_endpoint="Chat Flow",
    config_overrides=overrides_config,
)

Mounted a54b486b-8066-4c7c-9d34-4c44fedf82df at flows:Chat Flow:mounts:local:a54b486b-8066-4c7c-9d34-4c44fedf82df


In [21]:
input_message1 = FlowMessage(
    data={"id": 0, "question": "The capital of Switzerland is [[insert capital]]."},
)

input_message2 = FlowMessage(
    data={
        "id": 0,
        "query": "[[Write back Margaret as an email and thank her for the pertinant questions ]]. The translation of 'Ciao come stai ?' \
            is: [[insert translation and explain the meaning of each word seperately]]. [[Sign of with my name: Nicolas Baldwin]]"
    },
)

messages = [input_message1, input_message2]

for msg in messages:
    print("~~~~Sent message~~~~\n", msg.data)
    future = personalized_chatbot.get_reply_future(msg)
    reply = future.get_data()
    print("~~~~Reply~~~~ \n",reply["api_output"])

~~~~Sent message~~~~
 {'id': 0, 'question': 'The capital of Switzerland is [[insert capital]].'}


~~~~Reply~~~~ 
 The capital of Switzerland is Bern.
~~~~Sent message~~~~
 {'id': 0, 'query': "[[Write back Larry as an email and thank him for the pertinant questions ]]. The translation of 'Ciao come stai ?'             is: [[insert translation and explain the meaning of each word seperately]]. [[Sign of with my name: Nicolas Baldwin]]"}
~~~~Reply~~~~ 
 Subject: Thank You for Your Pertinent Questions

Hi Larry,

I wanted to express my gratitude for your pertinent questions. Your engagement is truly appreciated.

The translation of 'Ciao come stai?' is as follows:

- 'Ciao' means 'hello' or 'hi'.
- 'Come' means 'how'.
- 'Stai' is the informal second-person singular form of the verb 'stare', which means 'are you'.

So, 'Ciao come stai?' translates to 'Hello, how are you?' in English.

Thank you once again for your questions.

Best regards,
Nicolas Baldwin
